In [5]:
import numpy as np
import matplotlib.pyplot as plt

from utils_bayesian import PriorUniform
from mcmc import TMCMCCalibrator

# --- Toy forward model & likelihood ---
# y = θ1 + 0.5*θ2 + ε, ε ~ N(0, σ^2)
sigma = 0.3
def forward(theta):  # (n,2)->(n,)
    theta = np.atleast_2d(theta)
    return theta[:,0] + 0.5*theta[:,1]

def loglik(samples, y_obs):  # vectorized
    mu = forward(samples)
    return -0.5*((y_obs - mu)/sigma)**2 - 0.5*np.log(2*np.pi*sigma**2)

# --- Ground truth & observation ---
theta_true = np.array([1.5, -0.5])
y_obs = theta_true[0] + 0.5*theta_true[1] + sigma*np.random.randn()

# --- Priors: θ ∈ [-5,5]^2 ---
prior = PriorUniform(lb=np.array([-5., -5.]), ub=np.array([5., 5.]))

# --- TMCMC calibration ---
cal = TMCMCCalibrator(n_particles=3000, n_mcmc_steps=4, max_mcmc_steps=6,
                      target_ess_frac=0.95, adapt_scale=True, store_trace=False)

cal.setup(priors=[prior], log_likelihood=loglik)
post = cal.calibrate(observations=y_obs)  # returns dict

samples = post["samples"]
print("Posterior mean:", samples.mean(axis=0))
print("True θ:", theta_true)

plt.figure(figsize=(6,5))
plt.scatter(samples[:,0], samples[:,1], s=3, alpha=0.25, label="TMCMC samples")
plt.scatter(theta_true[0], theta_true[1], c="r", marker="x", s=100, label="θ_true")
plt.xlabel("θ1"); plt.ylabel("θ2"); plt.grid(True); plt.legend()
plt.title("TMCMC posterior (toy linear model)")
plt.show()


ValueError: could not broadcast input array from shape (3000,2) into shape (3000,)